In [1]:
import time
import threading
from datetime import datetime
from typing import Dict, Optional
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [4]:
class TradingApp(EClient, EWrapper):
    def __init__(self)-> None:
        EWrapper.__init__(self)
        EClient.__init__(self, self)
        self.data: Dict[int, pd.DataFrame] = {}
        self.nextOrderId = None # initialize nextOrderId

    def error(self, reqId: int, errorCode: int, errorString: str, advanced: any) -> None:
        print(f"Error: {reqId}, {errorCode}, {errorString}")

    def nextValidId(self, orderId: int) -> None:
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:
        self.data[reqId] = pd.DataFrame(columns = ['time', 'high', 'low', 'close', 'volume'])
        self.data[reqId].set_index('time', inplace = True)
        self.reqHistoricalData(
            reqId = reqId,
            contract  = contract,
            endDateTime = '',
            durationStr = '1 D',
            barSizeSetting = '1 min',
            whatToShow = 'TRADES', #try switching to TRADES data and get volume data
            useRTH = 0,
            formatDate = 2,
            keepUpToDate = False,
            chartOptions = [],
        )
        time.sleep(3)
        return self.data[reqId]

    def historicalData(self, reqId: int, bar: BarData) -> None:
        df = self.data[reqId]
        df.loc[
            pd.to_datetime(bar.date, unit = 's'),
            ['high', 'low', 'close', 'volume']
        ] = [bar.high, bar.low, bar.close, bar.volume]
        df = df.astype(float)
        self.data[reqId] = df

    @staticmethod
    def get_stock_contract(symbol:str) -> Contract:
        contract = Contract()
        contract.symbol = symbol
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        return contract

    def place_order(self, contract: Contract, action: str, order_type: str, quantity: int) -> None:
        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity

        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1
        print('Order Placed')

In [9]:
app = TradingApp()

app.connect('127.0.0.1' , 7497, clientId = 2) #ip address, port ip, clientId = 1

threading.Thread(target = app.run, daemon = True).start()

while True:
    if isinstance(app.nextOrderId, int):
        print('Connected')
        break
    else:
        print('waiting for connection')
        time.sleep(2)

waiting for connection
Error: -1, 2104, Market data farm connection is OK:usfarm.nj
Error: -1, 2104, Market data farm connection is OK:usfarm
Error: -1, 2106, HMDS data farm connection is OK:euhmds
Error: -1, 2106, HMDS data farm connection is OK:fundfarm
Error: -1, 2106, HMDS data farm connection is OK:ushmds
Error: -1, 2158, Sec-def data farm connection is OK:secdefnj
Connected


In [16]:
nvda = TradingApp.get_stock_contract('NVDA')
data = app.get_historical_data(99, nvda)

data.iloc[]
# reqId is arbitrary, 99 is used as an example

,high,low,close,volume
time,,,,
2025-04-01 20:19:00,110.17,110.10,110.13,14456.0
2025-04-01 20:20:00,110.13,110.06,110.08,17906.0
2025-04-01 20:21:00,110.08,109.95,110.00,34206.0
2025-04-01 20:22:00,110.00,109.92,109.95,24107.0
2025-04-01 20:23:00,109.98,109.93,109.98,8148.0


In [30]:
app.disconnect()

In [6]:
import pandas as pd
import numpy as np

def SMA(df: pd.DataFrame, period: int): #Example of SMA
    df['SMA'] = df['close'].rolling(window = period).mean()

def EMA(df: pd.DataFrame, period: int):
    df[f'{period}-day EMA'] = df['close'].ewm(span = period, adjust = False, min_periods = period).mean()
    #print(df)

def MACD(df: pd.DataFrame, short_ema: int, long_ema: int, signal_ema: int):
    EMA(df, short_ema)
    EMA(df, long_ema)
    EMA(df, signal_ema)
    df['MACD_line'] = df[f'{short_ema}-day EMA'] - df[f'{long_ema}-day EMA']

def full_SO(df: pd.DataFrame, lookback_period: int, smooth_period: int): # LL = lowest low, HH = highest high
    df['LL'] = df['low'].rolling(window = lookback_period).min()
    df['HH'] = df['high'].rolling(window = lookback_period).max()
    df['Fast %K'] = (df['close'] - df['LL']) / (df['HH'] - df['LL']) * 100
    df['Full %K'] = df['Fast %K'].rolling(window = smooth_period).mean()
    df['Full %D'] = df['Full %K'].rolling(window = smooth_period).mean()

def ATR(df: pd.DataFrame, period: int): # fix later maybe: annoying!
    df['previous low'] = df['low'].shift(1)
    df['TR'] = np.maximum(df['high'] - df['low'], np.abs(df['high'] - df['previous low']), np.abs(df['low'] - df['previous low']))
    
def Chaikin_OS(df: pd.DataFrame, period: int):
    df['Money Flow Multiplier'] = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'])
    df['Money Flow Volume'] = df['Money Flow Multiplier'] * df['volume']
    df['prev_MFV'] = df['Money Flow Volume'].shift(1)
    df['ADL'] = df['prev_MFV'] + df['Money Flow Volume']
    df['3 day EMA'] = df['ADL'].ewm(span = 3, adjust = False, min_periods = 3).mean()
    df['10 day EMA'] = df['ADL'].ewm(span = 10, adjust = False, min_periods = 10).mean()
    df['chaikin oscillator'] = df['3 day EMA'] - df['10 day EMA']

In [21]:
# Generate a datetime index (e.g., 100 minutes starting from now)
time_index = pd.date_range(start='2023-01-01 09:30', periods=100, freq='min')  # 'T' means minutely frequency

# Generate random stock data
np.random.seed(42)  # for reproducibility
close = np.random.normal(loc=100, scale=5, size=100) # simulate closing prices
high = close + np.random.uniform(0.5, 10, size=100)  # high is a bit higher than close
low = close - np.random.uniform(0.5, 10, size=100)   # low is a bit lower than close
volume = np.random.randint(1000, 10000, size=100)     # random volumes

# Create the DataFrame
data = pd.DataFrame({
    'high': high,
    'low': low,
    'close': close,
    'volume': volume
}, index=time_index)

data.index.name = 'Time'

period = 14
smooth_per = 3
short_ema = 12
long_ema = 24
signal_ema = 9

macd = MACD(data, short_ema, long_ema, signal_ema)
stoch_os = full_SO(data, period, smooth_per)
chaikin = Chaikin_OS(data, period)

buy_signal_bool = (data['MACD_line'] > 0) & (data[f'{short_ema}-day EMA'] > data[f'{long_ema}-day EMA']) & (data['Full %K'] > data['Full %D']) \
             & (data['Full %K'] > 0) & (data['Full %D'] > 0) & (data['chaikin oscillator'] > 0)

print(not buy_signal_bool.shift(1).iloc[-1])

True


In [4]:
period = 14
smooth_per = 3
short_ema = 12
long_ema = 24
signal_ema = 9

# EX for a contract:
nvda = TradingApp.get_stock_contract('NVDA')
# not using instance method of our app because get_stock_contract is a staticmethod

while True:
    # get data for contract
    data = app.get_historical_data(99, nvda)

    if len(data) < period:
        print(f'There are only {len(data)} entries of data, skipping:')
        continue

    # compute indicators
    macd = MACD(data, short_ema, long_ema, signal_ema)
    stoch_os = full_SO(data, period, smooth_per)
    chaikin = Chaikin_OS(data, period)

    # # check latest indicator values
    # last_macd = macd.iloc[-1]
    # last_stoch = stoch_os.iloc[-1]
    # last_chaikin = chaikin.iloc[-1]
    
    # # latest indicator values
    # last_macd_val = last_macd['MACD_line']
    # last_macd_short = last_macd[f'{short_ema}-day EMA']
    # last_macd_long = last_macd[f'{long_ema}-day EMA']
    # stoch_full_k = last_stoch['Full %K']
    # stoch_full_d = last_stoch['Full %D']
    # last_chaikin_val = last_chaikin['chaikin oscillator']
    # #check if last_macd_val produces number and not dataframe

    # #check second lastest indicator values too see if there is a crossover
    # cross_macd = macd.iloc[-2]
    # cross_stoch = stoch_os.iloc[-2]
    # cross_chaikin = chaikin.iloc[-2]

    # crossover signal to see if trading signal has changed
    buy_crossover_signals = (data['MACD_line'] > 0) & (data[f'{short_ema}-day EMA'] > data[f'{long_ema}-day EMA']) & (data['Full %K'] > data['Full %D']) \
             & (data['Full %K'] > 0) & (data['Full %D'] > 0) & (data['chaikin oscillator'] > 0)

    sell_crossover_signals = (data[f'{short_ema}-day EMA'] < data[f'{long_ema}-day EMA']) & (data['Full %K'] < data['Full %D']) \
             & (data['Full %K'] < 70) & (data['Full %D'] < 70) & (data['chaikin oscillator'] < 0)
    
    if (buy_crossover_signal_bool.iloc[-1]) and (not buy_crossover_signal_bool.shift(1).iloc[-1]):
        print('Entry signal: Buy')
        app.place_order(nvda, 'BUY', 'MKT', 10)

    elif (sell_crossover_signal_bool.iloc[-1]) and (not sell_crossover_signal_bool.shift(1).iloc[-1]):
        print('Exit signal: Sell')
        app.place_order(nvda, 'Sell', 'MKT', 10)



NameError: name 'TradingApp' is not defined